In [1]:
import re

import numpy as np
import pandas as pd

In [2]:
df_2017 = pd.read_csv('data/survey/2017.csv')
df_2018 = pd.read_csv('data/survey/2018.csv')
df_2019 = pd.read_csv('data/survey/2019.csv')
df_2020 = pd.read_csv('data/survey/2020.csv')
df_2021 = pd.read_csv('data/survey/2021.csv')
df_2022 = pd.read_csv('data/survey/2022.csv')
surveys = [df_2017, df_2018, df_2019, df_2020, df_2021, df_2022]

/var/folders/3y/tkr55n5x48x9_915tg6kwnch0000gp/T/ipykernel_88707/942721973.py:2: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2018 = pd.read_csv('data/survey/2018.csv')


In [228]:
clean_2017 = pd.DataFrame()
missing_2017 = pd.Series((np.nan for _ in range(len(df_2017))))

clean_2017['Year'] = pd.Series((2017 for _ in range(len(df_2017))))


def cap_salary(s: float):
    if np.isnan(s) or s > 250_000:
        return np.nan
    return s


clean_2017['Salary'] = df_2017['Salary'].apply(cap_salary)
clean_2017['RespondentType'] = df_2017['Professional'].replace(
    ['Student', 'Professional developer', 'Professional non-developer who sometimes writes code',
     'Used to be a professional developer', 'None of these'],
    ['stu', 'dev', 'non-dev', 'ex-dev', np.nan],
)
clean_2017['Age'] = missing_2017
clean_2017['Employment'] = df_2017['EmploymentStatus'].replace(
    ['Not employed, and not looking for work', 'Employed full-time', 'Employed part-time',
     'Independent contractor, freelancer, or self-employed', 'Not employed, but looking for work',
     'I prefer not to say', 'Retired'],
    ['notwork', 'fulltime', 'parttime', 'independent', 'searching', np.nan, 'retired'],
)
clean_2017['CodingActivities'] = df_2017['ProgramHobby'].replace(
    ['Yes, both', 'Yes, I program as a hobby', 'No', 'Yes, I contribute to open source projects'],
    ['Hobby;Contribute to open-source projects', 'Hobby', np.nan, 'Contribute to open-source projects'],
)
clean_2017['Education'] = df_2017['FormalEducation'].replace(
    ['Secondary school', "Some college/university study without earning a bachelor's degree", "Bachelor's degree",
     'Doctoral degree', "Master's degree", 'Professional degree', 'Primary/elementary school', 'I prefer not to answer',
     'I never completed any formal education'],
    ['secondary', 'tertiary', 'bachelor', 'doctor', 'master', 'professional', 'primary', np.nan, 'none'],
)

clean_2017['OrgSize'] = df_2017['CompanySize'].replace(
    ['Fewer than 10 employees', "I don't know", 'I prefer not to answer'],
    ['2 to 9 employees', np.nan, np.nan],
)


def filter_years(s: str | float) -> str | int:
    if isinstance(s, float):
        return s
    if s == 'Less than a year':
        return 0
    return int(s.split()[0])


clean_2017['YearsCode'] = df_2017['YearsProgram'].apply(filter_years)

clean_2017['YearsCodePro'] = df_2017['YearsCodedJob'].apply(filter_years).combine_first(
    df_2017['YearsCodedJobPast'].apply(filter_years))


def replace_multi(rep: dict[str, str]):
    def _replace_multi(s: str | float):
        if not isinstance(s, str):
            return s
        for original, new in rep.items():
            s = s.replace(original, new).strip('; ')
        if s:
            return re.sub(';+', ';', s)
        return np.nan

    return _replace_multi


def copy_replace(target: str):
    def _copy_replace(a: str | float, b: str | float):
        if isinstance(b, str):
            return a.replace(target, b)
        return a

    return _copy_replace


def remove_spaces(s: str | float):
    if not isinstance(s, str):
        return s
    return re.sub(r' ?; ?', ';', s)


clean_2017['DevType'] = df_2017['DeveloperType'].combine_first(df_2017['NonDeveloperType']).combine(
    df_2017['WebDeveloperType'], copy_replace('Web developer')).apply(remove_spaces).apply(replace_multi({
    'Other': '',
    'Mobile developer': 'Developer, mobile',
    'Full stack Web developer': 'Developer, full-stack',
    'Back-end Web developer': 'Developer, back-end',
    'Front-end Web developer': 'Developer, front-end',
    'Web developer': 'Developer, full-stack',
    'Desktop applications developer': 'Developer, desktop or enterprise applications',
    'Data scientist': 'Data scientist or machine learning specialist',
    'Machine learning specialist': 'Data scientist or machine learning specialist',
    'Developer with a statistics or mathematics background': '',
    'Database administrator': 'Data scientist or machine learning specialist',
    'Graphics programming': 'Developer, game or graphics',
    'Quality assurance engineer': 'Developer, QA or test',
    'Graphic designer': 'Designer',
    'C-suite executive': 'Senior Executive (C-Suite, VP, etc.)',
    'Product manager': 'Product manager',
    'Designer or illustrator': 'Designer',
    'Marketing or sales manager': 'Marketing or sales professional',
    'Embedded applications/devices developer': 'Developer, embedded applications or devices',
    'Systems administrator': 'System administrator',
    'Analyst or consultant': 'Data or business analyst',
    'Elected official': ''
}))

clean_2017['JobSat'] = df_2017['JobSatisfaction'].apply(lambda s: max(min(np.round((s + 1) / 2), 5), 1))

clean_2017['JobSeek'] = df_2017['JobSeekingStatus'].replace(
    ['I am actively looking for a job', 'I am not interested in new job opportunities',
     "I'm not actively looking, but I am open to new opportunities"],
    ['yes', 'no', 'maybe']
)
clean_2017['LastNewJob'] = df_2017['LastNewJob'].replace(['Not applicable/ never'], [np.nan])

clean_2017['Currency'] = df_2017['Currency']

clean_2017['LearnCodeFrom'] = df_2017['EducationTypes'].apply(remove_spaces).apply(replace_multi({
    'Online course': 'Online Courses or Certification',
    'Self-taught': '',
    'Part-time/evening course': 'School (i.e., University, College, etc)',
    'On-the-job training': 'On the job training',
    'Industry certification': 'Online Courses or Certification',
    'Bootcamp': 'Coding Bootcamp',
    'Coding competition': 'Other online resources (e.g., videos, blogs, forum)',
    'Open source contributions': 'Other online resources (e.g., videos, blogs, forum)',
    'Hackathon': 'Hackathons (virtual or in-person)',
}))

clean_2017['LangPresent'] = df_2017['HaveWorkedLanguage'].apply(remove_spaces).apply(
    replace_multi({'Matlab': 'MATLAB'}))
clean_2017['LangFuture'] = df_2017['WantWorkLanguage'].apply(remove_spaces).apply(replace_multi({'Matlab': 'MATLAB'}))

clean_2017['WebframePresent'] = df_2017['HaveWorkedFramework'].apply(remove_spaces).apply(replace_multi({
    'AngularJS': 'Angular/Angular.js',
    'Angular': 'Angular/Angular.js',
}))
clean_2017['WebframeFuture'] = df_2017['WantWorkFramework'].apply(remove_spaces).apply(replace_multi({
    'AngularJS': 'Angular.js'
}))

clean_2017['DatabasePresent'] = df_2017['HaveWorkedDatabase'].apply(remove_spaces)
clean_2017['DatabaseFuture'] = df_2017['WantWorkDatabase'].apply(remove_spaces)

clean_2017['PlatformPresent'] = df_2017['HaveWorkedPlatform'].apply(remove_spaces)
clean_2017['PlatformFuture'] = df_2017['WantWorkPlatform'].apply(remove_spaces)

clean_2017['CodeEditor'] = df_2017['IDE'].apply(remove_spaces).apply(replace_multi({
    'IPython / Jupyter': 'IPython/Jupyter',
    'PHPStorm': 'PhpStorm'
}))

clean_2017['VersionControlSystem'] = df_2017['VersionControl'].apply(remove_spaces).apply(replace_multi({
    'Subversion': 'SVN',
    'Zip file back-ups': 'other',
    'Team Foundation Server': 'other',
    'I use some other system': 'other',
    'Visual Source Safe': 'other',
    'Copying and pasting files to network shares': 'other',
    'Rational ClearCase': 'other',
    "I don't use version control": '',
}))

clean_2017['Gender'] = df_2017['Gender'].apply(replace_multi({
    'Male': 'Man',
    'Female': 'Woman',
    'Gender non-conforming': '',
    'Other': '',
    'Transgender': '',
    ' ': '',
    'Man;Woman': '',
}))

clean_2017['Student'] = df_2017['University'].replace(
    ['No', 'Yes, full-time', 'Yes, part-time', 'I prefer not to say'],
    ['no', 'yes', 'parttime', np.nan]
)
countries = {
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'United States of America': 'United States',
    'Virgin Islands (USA)': 'Virgin Islands',
    'Viet Nam': 'Vietnam',
    'Venezuela, Bolivarian Republic of...': 'Venezuela',
    'North Korea': 'Korea',
    'South Korea': 'Korea',
    'Republic of Korea': 'Korea',
    "Democratic People's Republic of Korea": 'Korea',
    'Slovak Republic': 'Slovakia',
    'New Caledonia (French)': 'New Caledonia',
    'Martinique (French)': 'Martinique',
    'Iran, Islamic Republic of...': 'Iran',
    'Hong Kong (S.A.R.)': 'Hong Kong',
    'Congo, Republic of the...': 'Congo',
    'Democratic Republic of the Congo': 'Congo',
    "Lao People's Democratic Republic": 'Laos',
    "Côte d'Ivoire": 'Ivory Coast',
    "Ivory Coast (Cote D'Ivoire)": 'Ivory Coast',
    'The former Yugoslav Republic of Macedonia': 'Macedonia',
    'I prefer not to say': np.nan,
    'Other Country (Not Listed Above)': np.nan,
    'Azerbaidjan': 'Azerbaijan',
    'Bosnia-Herzegovina': 'Bosnia and Herzegovina',
    'Libyan Arab Jamahiriya': 'Libya',
    'Saint Vincent & Grenadines': 'Saint Vincent and the Grenadines',
    'Syrian Arab Republic': 'Syria',
    'United Republic of Tanzania': 'Tanzania'
}

clean_2017['Country'] = df_2017['Country'].replace(countries)

clean_2017.head()

,Year,Salary,RespondentType,Age,Employment,CodingActivities,Education,OrgSize,YearsCode,YearsCodePro,...,WebframeFuture,DatabasePresent,DatabaseFuture,PlatformPresent,PlatformFuture,CodeEditor,VersionControlSystem,Gender,Student,Country
0,2017,NaN,stu,NaN,notwork,Hobby;Contribute to open-source projects,secondary,NaN,2.0,NaN,...,NaN,NaN,NaN,iOS,iOS,Atom;Xcode,NaN,Man,no,United States
1,2017,NaN,stu,NaN,parttime,Hobby;Contribute to open-source projects,tertiary,20 to 99 employees,9.0,NaN,...,.NET Core,MySQL;SQLite,MySQL;SQLite,Amazon Web Services (AWS),Linux Desktop;Raspberry Pi;Amazon Web Services...,Atom;Notepad++;Vim;PyCharm;RubyMine;Visual Stu...,Git,Man,yes,United Kingdom
2,2017,113750.0,dev,NaN,fulltime,Hobby;Contribute to open-source projects,bachelor,"10,000 or more employees",20.0,20.0,...,NaN,MySQL,NaN,NaN,NaN,Sublime Text;Vim;IntelliJ,Mercurial,Man,no,United Kingdom
3,2017,NaN,non-dev,NaN,fulltime,Hobby;Contribute to open-source projects,doctor,"10,000 or more employees",14.0,9.0,...,Hadoop;Node.js;React,MongoDB;Redis;SQL Server;MySQL;SQLite,MongoDB;Redis;SQL Server;MySQL;SQLite,Windows Desktop;Linux Desktop;Mac OS;Amazon We...,Windows Desktop;Linux Desktop;Mac OS;Amazon We...,Notepad++;Sublime Text;TextMate;Vim;IPython/Ju...,Git,Man,no,United States
4,2017,NaN,dev,NaN,fulltime,Hobby,master,10 to 19 employees,20.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,Switzerland


In [229]:
clean_2018 = pd.DataFrame()


def merge(a: str, b: str) -> str:
    return f'{a};{b}'.strip(';')


clean_2018['Year'] = pd.Series((2018 for _ in range(len(df_2018))))

clean_2018['Salary'] = df_2018['ConvertedSalary'].apply(cap_salary)
clean_2018['CodingActivities'] = df_2018['Hobby'].replace(
    ['Yes', 'No'], ['Hobby', '']
).combine(df_2018['OpenSource'].replace(
    ['Yes', 'No'], ['Contribute to open-source projects', '']
), merge, fill_value='').replace([''], [np.nan])

clean_2018['Country'] = df_2018['Country'].replace(countries)
clean_2018['Student'] = df_2018['Student'].replace(
    ['No', 'Yes, full-time', 'Yes, part-time'],
    ['no', 'yes', 'parttime']
)
clean_2018['Age'] = df_2018['Age'].replace(
    ['25 - 34 years old', '35 - 44 years old', '18 - 24 years old',
     '45 - 54 years old', '55 - 64 years old', 'Under 18 years old',
     '65 years or older'],
    ['25-34', '35-44', '18-24', '45-54', '55-64', '-17', '65-']
)

clean_2018['Employment'] = df_2018['Employment'].replace(
    ['Not employed, and not looking for work', 'Employed full-time', 'Employed part-time',
     'Independent contractor, freelancer, or self-employed', 'Not employed, but looking for work',
     'I prefer not to say', 'Retired'],
    ['notwork', 'fulltime', 'parttime', 'independent', 'searching', np.nan, 'retired'],
)

clean_2018['Education'] = df_2018['FormalEducation'].replace(
    ['Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
     "Some college/university study without earning a degree", "Bachelor’s degree (BA, BS, B.Eng., etc.)",
     'Other doctoral degree (Ph.D, Ed.D., etc.)', "Master’s degree (MA, MS, M.Eng., MBA, etc.)",
     'Professional degree (JD, MD, etc.)', 'Primary/elementary school', 'I prefer not to answer',
     'I never completed any formal education', 'Associate degree'],
    ['secondary', 'tertiary', 'bachelor', 'doctor', 'master', 'professional', 'primary', np.nan, 'none', 'assoc'],
)

clean_2018['OrgSize'] = df_2018['CompanySize'].replace(
    ['Fewer than 10 employees', "I don't know", 'I prefer not to answer'],
    ['2 to 9 employees', np.nan, np.nan],
)

clean_2018['DevType'] = df_2018['DevType'].apply(replace_multi({
    'Game or graphics developer': 'Developer, game or graphics',
    'Embedded applications or devices developer': 'Developer, embedded applications or devices',
    'QA or test developer': 'Developer, QA or test',
    'Back-end developer': 'Developer, back-end',
    'Full-stack developer': 'Developer, full-stack',
    'C-suite executive (CEO, CTO, etc.)': 'Senior Executive (C-Suite, VP, etc.)',
    'Educator or academic researcher': 'Educator or academic',
    'Desktop or enterprise applications developer': 'Developer, desktop or enterprise applications',
    'Front-end developer': 'Developer, front-end',
    'Mobile developer': 'Developer, mobile',
}))


def average_year(val: str | float) -> float:
    if not isinstance(val, str):
        return val
    digits = [int(d) for d in re.findall(r'\d+', val)]
    return sum(digits) / len(digits)


clean_2018['YearsCode'] = df_2018['YearsCoding'].apply(average_year)
clean_2018['YearsCodePro'] = df_2018['YearsCodingProf'].apply(average_year)

clean_2018['JobSat'] = df_2018['JobSatisfaction'].replace(
    ['Extremely dissatisfied', 'Moderately dissatisfied', 'Slightly dissatisfied', 'Neither satisfied nor dissatisfied',
     'Slightly satisfied', 'Moderately satisfied', 'Extremely satisfied'],
    [1, 2, 2, 3, 4, 4, 5]
)
clean_2018['JobSeek'] = df_2018['JobSearchStatus'].replace(
    ['I am actively looking for a job', 'I am not interested in new job opportunities',
     "I’m not actively looking, but I am open to new opportunities"],
    ['yes', 'no', 'maybe']
)

clean_2018['LastNewJob'] = df_2018['LastNewJob'].replace(["I've never had a job"], [np.nan])
clean_2018['Currency'] = df_2018['Currency']

clean_2018['CodeEditor'] = df_2018['IDE'].apply(replace_multi({
    'IPython / Jupyter': 'IPython/Jupyter',
    'PHPStorm': 'PhpStorm'
}))

clean_2018['LangPresent'] = df_2018['LanguageWorkedWith'].apply(replace_multi({
    'Matlab': 'MATLAB',
    'Delphi/Object Pascal': 'Delphi',
    'HTML': 'HTML/CSS'
}))
clean_2018['LangFuture'] = df_2018['LanguageDesireNextYear'].apply(replace_multi({
    'Matlab': 'MATLAB',
    'Delphi/Object Pascal': 'Delphi',
    'HTML': 'HTML/CSS'
}))

clean_2018['WebframePresent'] = df_2018['FrameworkWorkedWith'].apply(replace_multi({
    'Hadoop': '',
    'Spark': '',
    'TensorFlow': '',
    'Torch/PyTorch': '',
    'Angular.js': 'Angular/Angular.js',
    'Angular': 'Angular/Angular.js',
}))
clean_2018['WebframeFuture'] = df_2018['FrameworkDesireNextYear'].apply(replace_multi({
    'Hadoop': '',
    'Spark': '',
    'TensorFlow': '',
    'Torch/PyTorch': '',
}))

clean_2018['DatabasePresent'] = df_2018['DatabaseWorkedWith'].apply(replace_multi({
    'IBM Db2': 'IBM DB2',
    'SQL Server': 'Microsoft SQL Server',
}))
clean_2018['DatabaseFuture'] = df_2018['DatabaseDesireNextYear'].apply(replace_multi({
    'IBM Db2': 'IBM DB2',
    'SQL Server': 'Microsoft SQL Server',
}))

clean_2018['PlatformPresent'] = df_2018['PlatformWorkedWith']
clean_2018['PlatformFuture'] = df_2018['PlatformDesireNextYear']

clean_2018['Gender'] = df_2018['Gender'].apply(replace_multi({
    'Male': 'Man',
    'Female': 'Woman',
    'Non-binary, genderqueer, or gender non-conforming': '',
    'Transgender': '',
    'Woman;Man': '',
}))

clean_2018['VersionControlSystem'] = df_2018['VersionControl'].apply(replace_multi({
    'Subversion': 'SVN',
    'Zip file back-ups': 'other',
    'Team Foundation Version Control': 'other',
    'I use some other system': 'other',
    'Copying and pasting files to network shares': 'other',
    "I don't use version control": '',
}))

clean_2018['LearnCodeFrom'] = df_2018['EducationTypes'].apply(replace_multi({
    'Other': '',
    'Completed an industry certification program (e.g. MCPD)': 'Online Courses or Certification',
    'Contributed to open source software': 'Other online resources (e.g., videos, blogs, forum)',
    'Participated in a full-time developer training program or bootcamp': 'Coding Bootcamp',
    'Participated in a hackathon': 'Hackathons (virtual or in-person)',
    'Participated in online coding competitions (e.g. HackerRank, CodeChef, TopCoder)': 'Other online resources (e.g., videos, blogs, forum)',
    'Received on-the-job training in software development': 'On the job training',
    'Taken a part-time in-person course in programming or software development': 'School (i.e., University, College, etc)',
    'Taken an online course in programming or software development (e.g. a MOOC)': 'Online Courses or Certification',
    'Taught yourself a new language, framework, or tool without taking a formal course': 'Other online resources (e.g., videos, blogs, forum)',
}))


def determine_respondent_type(dev: str | float, student: str | float):
    if isinstance(dev, str):
        if 'Developer' in dev:
            return 'dev'
    if not isinstance(student, str) or student == 'no':
        return 'non-dev'
    return 'stu'


clean_2018['RespondentType'] = clean_2018['DevType'].combine(clean_2018['Student'], determine_respondent_type)

clean_2018.head()

,Year,Salary,CodingActivities,Country,Student,Age,Employment,Education,OrgSize,DevType,...,WebframePresent,WebframeFuture,DatabasePresent,DatabaseFuture,PlatformPresent,PlatformFuture,Gender,VersionControlSystem,LearnCodeFrom,RespondentType
0,2018,NaN,Hobby,Kenya,no,25-34,parttime,bachelor,20 to 99 employees,"Developer, full-stack",...,Django;React,Django;React,Redis;Microsoft SQL Server;MySQL;PostgreSQL;Am...,Redis;Microsoft SQL Server;MySQL;PostgreSQL;Am...,AWS;Azure;Linux;Firebase,AWS;Azure;Linux;Firebase,Man,Git,"Other online resources (e.g., videos, blogs, f...",dev
1,2018,70841.0,Hobby;Contribute to open-source projects,United Kingdom,no,35-44,fulltime,bachelor,"10,000 or more employees",Database administrator;DevOps specialist;Devel...,...,Django,React,Redis;PostgreSQL;Memcached,PostgreSQL,Linux,Linux,Man,Git;SVN,"Other online resources (e.g., videos, blogs, f...",dev
2,2018,NaN,Hobby;Contribute to open-source projects,United States,no,NaN,fulltime,assoc,20 to 99 employees,"Engineering manager;Developer, full-stack",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dev
3,2018,NaN,NaN,United States,no,35-44,fulltime,bachelor,100 to 499 employees,"Developer, full-stack",...,NaN,Angular;.NET Core;React,"Microsoft SQL Server;Microsoft Azure (Tables, ...","Microsoft SQL Server;Microsoft Azure (Tables, ...",Azure,Azure,Man,Git,Online Courses or Certification;Other online r...,dev
4,2018,21426.0,Hobby,South Africa,parttime,18-24,fulltime,tertiary,"10,000 or more employees","Data or business analyst;Developer, desktop or...",...,NaN,NaN,Microsoft SQL Server;PostgreSQL;Oracle;IBM DB2,PostgreSQL;Oracle;IBM DB2,Arduino;Windows Desktop or Server,Arduino;Windows Desktop or Server,Man,other,"School (i.e., University, College, etc);On the...",dev


In [230]:
clean_2019 = pd.DataFrame()

clean_2019['RespondentType'] = df_2019['MainBranch'].replace(
    ['I am a developer by profession', 'I am a student who is learning to code',
     'I am not primarily a developer, but I write code sometimes as part of my work', 'I code primarily as a hobby',
     'I used to be a developer by profession, but no longer am'],
    ['dev', 'stu', 'non-dev', 'non-dev', 'ex-dev']
)

clean_2019['Year'] = pd.Series((2019 for _ in range(len(df_2019))))
clean_2019['CodingActivities'] = df_2019['Hobbyist'].replace(
    ['Yes', 'No'], ['Hobby', '']
).combine(df_2019['OpenSourcer'].replace(
    ['Once a month or more often', 'Less than once a month but more than once per year', 'Less than once per year',
     'Never'], ['Contribute to open-source projects', '', '', '']
), merge, fill_value='').replace([''], [np.nan])


def age_19(num: float):
    if np.isnan(num):
        return np.nan
    if num < 18:
        return '-17'
    if num < 25:
        return '18-24'
    if num < 35:
        return '25-34'
    if num < 45:
        return '35-44'
    if num < 55:
        return '45-54'
    if num < 65:
        return '55-64'
    return '65-'


clean_2019['Age'] = df_2019['Age'].apply(age_19)

clean_2019['Employment'] = df_2019['Employment'].replace(
    ['Not employed, and not looking for work', 'Employed full-time', 'Employed part-time',
     'Independent contractor, freelancer, or self-employed', 'Not employed, but looking for work',
     'Retired'],
    ['notwork', 'fulltime', 'parttime', 'independent', 'searching', 'retired'],
)

clean_2019['Country'] = df_2019['Country'].replace(countries)

clean_2019['Student'] = df_2019['Student'].replace(
    ['No', 'Yes, full-time', 'Yes, part-time'],
    ['no', 'yes', 'parttime']
)

clean_2019['Education'] = df_2019['EdLevel'].replace(
    ['Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
     "Some college/university study without earning a degree", "Bachelor’s degree (BA, BS, B.Eng., etc.)",
     'Other doctoral degree (Ph.D, Ed.D., etc.)', "Master’s degree (MA, MS, M.Eng., MBA, etc.)",
     'Professional degree (JD, MD, etc.)', 'Primary/elementary school', 'I prefer not to answer',
     'I never completed any formal education', 'Associate degree'],
    ['secondary', 'tertiary', 'bachelor', 'doctor', 'master', 'professional', 'primary', np.nan, 'none', 'assoc'],
)

clean_2019['OrgSize'] = df_2019['OrgSize'].replace(
    ['2-9 employees'],
    ['2 to 9 employees'],
)

clean_2019['DevType'] = df_2019['DevType'].apply(replace_multi({
    'C-suite executive (CEO, CTO, etc.)': 'Senior Executive (C-Suite, VP, etc.)',
    'Senior executive/VP': 'Senior Executive (C-Suite, VP, etc.)',
    'Educator': 'Educator or academic',
    'Academic researcher': 'Educator or academic',
}))


def filter_year_19(s: str | float):
    if not isinstance(s, str):
        return s
    if s == 'Less than 1 year':
        return 0
    if s == 'More than 50 years':
        return 50
    return int(s)


clean_2019['YearsCode'] = df_2019['YearsCode'].apply(filter_year_19)
clean_2019['YearsCodePro'] = df_2019['YearsCodePro'].apply(filter_year_19)

clean_2019['JobSat'] = df_2019['JobSat'].replace(
    ['Very dissatisfied', 'Slightly dissatisfied', 'Neither satisfied nor dissatisfied', 'Slightly satisfied',
     'Very satisfied'],
    [1, 2, 3, 4, 5]
)

clean_2019['Currency'] = df_2019['CurrencyDesc']
clean_2019['Salary'] = df_2019['ConvertedComp'].apply(cap_salary)

clean_2019['CodeEditor'] = df_2019['DevEnviron'].apply(replace_multi({
    'IPython / Jupyter': 'IPython/Jupyter',
    'PHPStorm': 'PhpStorm'
}))

clean_2019['Gender'] = df_2019['Gender'].apply(replace_multi({
    'Male': 'Man',
    'Female': 'Woman',
    'Non-binary, genderqueer, or gender non-conforming': '',
    'Woman;Man': '',
}))

clean_2019['LastNewJob'] = df_2019['LastHireDate'].replace(
    ['1-2 years ago', '3-4 years ago', "I've never had a job", 'NA - I am an independent contractor or self employed'],
    ['Between 1 and 2 years ago', 'Between 2 and 4 years ago', np.nan, np.nan]
)


def add_school_19(main: str | float, edu: str):
    add = 'School (i.e., University, College, etc)' if edu in ['tertiary', 'bachelor', 'doctor', 'master',
                                                               'professional'] else ''
    if not isinstance(main, str):
        main = ''
    val = f'{main};{add}'.strip(';')
    return val if val else np.nan


clean_2019['LearnCodeFrom'] = df_2019['EduOther'].apply(remove_spaces).apply(replace_multi({
    'Completed an industry certification program (e.g. MCPD)': 'Online Courses or Certification',
    'Contributed to open source software': 'Other online resources (e.g., videos, blogs, forum)',
    'Participated in a full-time developer training program or bootcamp': 'Coding Bootcamp',
    'Participated in a hackathon': 'Hackathons (virtual or in-person)',
    'Participated in online coding competitions (e.g. HackerRank, CodeChef, TopCoder)': 'Other online resources (e.g., videos, blogs, forum)',
    'Received on-the-job training in software development': 'On the job training',
    'Taken a part-time in-person course in programming or software development': 'School (i.e., University, College, etc)',
    'Taken an online course in programming or software development (e.g. a MOOC)': 'Online Courses or Certification',
    'Taught yourself a new language, framework, or tool without taking a formal course': 'Other online resources (e.g., videos, blogs, forum)',
})).combine(clean_2019['Education'], add_school_19, fill_value='')

clean_2019['JobSeek'] = df_2019['JobSeek'].replace(
    ['I am actively looking for a job', 'I am not interested in new job opportunities',
     "I’m not actively looking, but I am open to new opportunities"],
    ['yes', 'no', 'maybe']
)

# VCS missing for 2019
missing_2019 = pd.Series((np.nan for _ in range(len(df_2019))))
clean_2019['VersionControlSystem'] = missing_2019

clean_2019['LangPresent'] = df_2019['LanguageWorkedWith'].apply(replace_multi({'Matlab': 'MATLAB', 'Other(s):': ''}))
clean_2019['LangFuture'] = df_2019['LanguageDesireNextYear'].apply(replace_multi({'Matlab': 'MATLAB', 'Other(s):': ''}))

clean_2019['WebframePresent'] = df_2019['WebFrameWorkedWith'].apply(replace_multi({
    'Angular.js': 'Angular/Angular.js',
    'Angular': 'Angular/Angular.js',

}))
clean_2019['WebframeFuture'] = df_2019['WebFrameDesireNextYear'].apply(replace_multi({
    'Angular.js': 'Angular/Angular.js',
    'Angular': 'Angular/Angular.js',
}))

clean_2019['DatabasePresent'] = df_2019['DatabaseWorkedWith'].apply(replace_multi({
    'IBM Db2': 'IBM DB2',
    'SQL Server': 'Microsoft SQL Server',
}))
clean_2019['DatabaseFuture'] = df_2019['DatabaseDesireNextYear'].apply(replace_multi({
    'IBM Db2': 'IBM DB2',
    'SQL Server': 'Microsoft SQL Server',
}))

clean_2019['PlatformPresent'] = df_2019['PlatformWorkedWith']
clean_2019['PlatformFuture'] = df_2019['PlatformDesireNextYear']

clean_2019.head()

,RespondentType,Year,CodingActivities,Age,Employment,Country,Student,Education,OrgSize,DevType,...,JobSeek,VersionControlSystem,LangPresent,LangFuture,WebframePresent,WebframeFuture,DatabasePresent,DatabaseFuture,PlatformPresent,PlatformFuture
0,stu,2019,Hobby,-17,notwork,United Kingdom,no,primary,NaN,NaN,...,NaN,NaN,HTML/CSS;Java;JavaScript;Python,C;C++;C#;Go;HTML/CSS;Java;JavaScript;Python;SQL,Django;Flask,Flask;jQuery,SQLite,MySQL,MacOS;Windows,Android;Arduino;Windows
1,stu,2019,NaN,18-24,searching,Bosnia and Herzegovina,yes,secondary,NaN,"Developer, desktop or enterprise applications;...",...,yes,NaN,C++;HTML/CSS;Python,C++;HTML/CSS;JavaScript;SQL,Django,Django,NaN,MySQL,Windows,Windows
2,non-dev,2019,Hobby,25-34,fulltime,Thailand,no,bachelor,100 to 499 employees,"Designer;Developer, back-end;Developer, front-...",...,maybe,NaN,HTML/CSS,Elixir;HTML/CSS,NaN,Other(s):,PostgreSQL,PostgreSQL,NaN,NaN
3,dev,2019,NaN,18-24,fulltime,United States,no,bachelor,100 to 499 employees,"Developer, full-stack",...,no,NaN,C;C++;C#;Python;SQL,C;C#;JavaScript;SQL,NaN,NaN,MySQL;SQLite,MySQL;SQLite,Linux;Windows,Linux;Windows
4,dev,2019,Hobby;Contribute to open-source projects,25-34,fulltime,Ukraine,no,bachelor,"10,000 or more employees","Educator or academic;Developer, desktop or ent...",...,no,NaN,C++;HTML/CSS;Java;JavaScript;Python;SQL;VBA,HTML/CSS;Java;JavaScript;SQL;WebAssembly,Django;Express;Flask;jQuery;React.js;Spring,Flask;jQuery;React.js;Spring,Couchbase;MongoDB;MySQL;Oracle;PostgreSQL;SQLite,Couchbase;Firebase;MongoDB;MySQL;Oracle;Postgr...,Android;Linux;MacOS;Slack;Windows,Android;Docker;Kubernetes;Linux;Slack


In [231]:
clean_2020 = pd.DataFrame()
missing_2020 = pd.Series((np.nan for _ in range(len(df_2020))))

clean_2020['Year'] = pd.Series((2020 for _ in range(len(df_2020))))
clean_2020['RespondentType'] = df_2020['MainBranch'].replace(
    ['I am a developer by profession', 'I am a student who is learning to code',
     'I am not primarily a developer, but I write code sometimes as part of my work', 'I code primarily as a hobby',
     'I used to be a developer by profession, but no longer am'],
    ['dev', 'stu', 'non-dev', 'non-dev', 'ex-dev']
)

clean_2020['CodingActivities'] = df_2020['Hobbyist'].replace(
    ['Yes', 'No'], ['Hobby', '']
)

clean_2020['Employment'] = df_2020['Employment'].replace(
    ['Not employed, and not looking for work', 'Employed full-time', 'Employed part-time',
     'Independent contractor, freelancer, or self-employed', 'Not employed, but looking for work',
     'Retired', 'Student'],
    ['notwork', 'fulltime', 'parttime', 'independent', 'searching', 'retired', 'notwork'],
)
clean_2020['Age'] = df_2020['Age'].apply(age_19)

clean_2020['Country'] = df_2020['Country'].replace(countries)

clean_2020['Student'] = missing_2020

clean_2020['Education'] = df_2020['EdLevel'].replace(
    ['Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
     "Some college/university study without earning a degree", "Bachelor’s degree (B.A., B.S., B.Eng., etc.)",
     'Other doctoral degree (Ph.D., Ed.D., etc.)', "Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",
     'Professional degree (JD, MD, etc.)', 'Primary/elementary school', 'I prefer not to answer',
     'I never completed any formal education', 'Associate degree (A.A., A.S., etc.)'],
    ['secondary', 'tertiary', 'bachelor', 'doctor', 'master', 'professional', 'primary', np.nan, 'none', 'assoc'],
)

clean_2020['OrgSize'] = df_2020['OrgSize'].replace(
    ['2-9 employees'],
    ['2 to 9 employees'],
)

clean_2020['DevType'] = df_2020['DevType'].apply(replace_multi({
    'Senior executive/VP': 'Senior Executive (C-Suite, VP, etc.)',
    'Educator': 'Educator or academic',
    'Academic researcher': 'Educator or academic',
}))

clean_2020['YearsCode'] = df_2020['YearsCode'].apply(filter_year_19)
clean_2020['YearsCodePro'] = df_2020['YearsCodePro'].apply(filter_year_19)

clean_2020['JobSat'] = df_2020['JobSat'].replace(
    ['Very dissatisfied', 'Slightly dissatisfied', 'Neither satisfied nor dissatisfied', 'Slightly satisfied',
     'Very satisfied'],
    [1, 2, 3, 4, 5]
)

clean_2020['Currency'] = df_2020['CurrencyDesc']
clean_2020['Salary'] = df_2020['ConvertedComp'].apply(cap_salary)

clean_2020['CodeEditor'] = missing_2020

clean_2020['Gender'] = df_2020['Gender'].apply(replace_multi({
    'Male': 'Man',
    'Female': 'Woman',
    'Non-binary, genderqueer, or gender non-conforming': '',
    'Woman;Man': '',
}))

clean_2020['LastNewJob'] = missing_2020

clean_2020['LearnCodeFrom'] = missing_2020

clean_2020['JobSeek'] = df_2020['JobSeek'].replace(
    ['I am actively looking for a job', 'I am not interested in new job opportunities',
     "I’m not actively looking, but I am open to new opportunities"],
    ['yes', 'no', 'maybe']
)

clean_2020['VersionControlSystem'] = missing_2020

clean_2020['LangPresent'] = df_2020['LanguageWorkedWith'].apply(replace_multi({'Matlab': 'MATLAB'}))
clean_2020['LangFuture'] = df_2020['LanguageDesireNextYear'].apply(replace_multi({'Matlab': 'MATLAB'}))

clean_2020['WebframePresent'] = df_2020['WebframeWorkedWith'].apply(replace_multi({
    'Angular.js': 'Angular/Angular.js',
    'Angular': 'Angular/Angular.js',
}))
clean_2020['WebframeFuture'] = df_2020['WebframeDesireNextYear'].apply(replace_multi({
    'Angular.js': 'Angular/Angular.js',
    'Angular': 'Angular/Angular.js',
}))

clean_2020['DatabasePresent'] = df_2020['DatabaseWorkedWith'].apply(replace_multi({
    'IBM Db2': 'IBM DB2',
}))
clean_2020['DatabaseFuture'] = df_2020['DatabaseDesireNextYear'].apply(replace_multi({
    'IBM Db2': 'IBM DB2',
}))

clean_2020['PlatformPresent'] = df_2020['PlatformWorkedWith']
clean_2020['PlatformFuture'] = df_2020['PlatformDesireNextYear']

clean_2020.head()

,Year,RespondentType,CodingActivities,Employment,Age,Country,Student,Education,OrgSize,DevType,...,JobSeek,VersionControlSystem,LangPresent,LangFuture,WebframePresent,WebframeFuture,DatabasePresent,DatabaseFuture,PlatformPresent,PlatformFuture
0,2020,dev,Hobby,independent,NaN,Germany,NaN,master,2 to 9 employees,"Developer, desktop or enterprise applications;...",...,no,NaN,C#;HTML/CSS;JavaScript,C#;HTML/CSS;JavaScript,ASP.NET;ASP.NET Core,ASP.NET Core,Elasticsearch;Microsoft SQL Server;Oracle,Microsoft SQL Server,Windows,Android;iOS;Kubernetes;Microsoft Azure;Windows
1,2020,dev,,fulltime,NaN,United Kingdom,NaN,bachelor,"1,000 to 4,999 employees","Developer, full-stack;Developer, mobile",...,no,NaN,JavaScript;Swift,Python;Swift,NaN,NaN,NaN,NaN,iOS,iOS;Kubernetes;Linux;MacOS
2,2020,non-dev,Hobby,NaN,NaN,Russian Federation,NaN,NaN,NaN,NaN,...,NaN,NaN,Objective-C;Python;Swift,Objective-C;Python;Swift,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,dev,Hobby,NaN,25-34,Albania,NaN,master,20 to 99 employees,NaN,...,maybe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,ex-dev,Hobby,fulltime,25-34,United States,NaN,bachelor,NaN,NaN,...,NaN,NaN,HTML/CSS;Ruby;SQL,Java;Ruby;Scala,Ruby on Rails,Django;Ruby on Rails,MySQL;PostgreSQL;Redis;SQLite,MySQL;PostgreSQL,AWS;Docker;Linux;MacOS;Windows,Docker;Google Cloud Platform;Heroku;Linux;Windows


In [221]:
clean_2021 = pd.DataFrame()
missing_2021 = pd.Series((np.nan for _ in range(len(df_2021))))

clean_2021['Year'] = pd.Series((2021 for _ in range(len(df_2021))))
clean_2021['RespondentType'] = df_2021['MainBranch'].replace(
    ['I am a developer by profession', 'I am a student who is learning to code',
     'I am not primarily a developer, but I write code sometimes as part of my work', 'I code primarily as a hobby',
     'I used to be a developer by profession, but no longer am', 'None of these'],
    ['dev', 'stu', 'non-dev', 'non-dev', 'ex-dev', 'non-dev']
)

clean_2021['CodingActivities'] = missing_2021

clean_2021['Employment'] = df_2021['Employment'].replace(
    ['Not employed, and not looking for work', 'Employed full-time', 'Employed part-time',
     'Independent contractor, freelancer, or self-employed', 'Not employed, but looking for work',
     'Retired', 'Student, full-time', 'Student, part-time', 'I prefer not to say'],
    ['notwork', 'fulltime', 'parttime', 'independent', 'searching', 'retired', 'notwork', 'parttime', np.nan],
)

clean_2021['Country'] = df_2021['Country'].replace(countries)


def is_student_21(emp: str):
    if emp == 'Student, part-time':
        return 'parttime'
    if emp == 'Student, full-time':
        return 'yes'
    return 'no'


clean_2021['Student'] = df_2021['Employment'].apply(is_student_21)
clean_2021['Age'] = df_2021['Age'].replace(
    ['Prefer not to say', '25-34 years old', '18-24 years old', '35-44 years old', '45-54 years old',
     'Under 18 years old', '65 years or older', '55-64 years old'],
    [np.nan, '25-34', '18-24', '35-44', '45-54', '-17', '65-', '55-64']
)

clean_2021['Education'] = df_2021['EdLevel'].replace(
    ['Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
     "Some college/university study without earning a degree", "Bachelor’s degree (B.A., B.S., B.Eng., etc.)",
     'Other doctoral degree (Ph.D., Ed.D., etc.)', "Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",
     'Professional degree (JD, MD, etc.)', 'Primary/elementary school', 'I prefer not to answer',
     'I never completed any formal education', 'Associate degree (A.A., A.S., etc.)', 'Something else'],
    ['secondary', 'tertiary', 'bachelor', 'doctor', 'master', 'professional', 'primary', np.nan, 'none', 'assoc',
     np.nan],
)

clean_2021['OrgSize'] = df_2021['OrgSize'].replace(
    ['I don’t know'],
    [np.nan],
)

clean_2021['DevType'] = df_2021['DevType'].apply(replace_multi({
    'Other (please specify):': '',
    'Senior executive/VP': 'Senior Executive (C-Suite, VP, etc.)',
    'Educator': 'Educator or academic',
    'Academic researcher': 'Educator or academic',
}))

clean_2021['YearsCode'] = df_2021['YearsCode'].apply(filter_year_19)
clean_2021['YearsCodePro'] = df_2021['YearsCodePro'].apply(filter_year_19)

clean_2021['JobSat'] = missing_2021

clean_2021['Currency'] = df_2021['Currency']
clean_2021['Salary'] = df_2021['ConvertedCompYearly'].apply(cap_salary)

clean_2021['CodeEditor'] = df_2021['NEWCollabToolsHaveWorkedWith']

clean_2021['Gender'] = df_2021['Gender'].apply(replace_multi({
    'Male': 'Man',
    'Female': 'Woman',
    'Non-binary, genderqueer, or gender non-conforming': '',
    'Or, in your own words:': '',
    'Prefer not to say': '',
    'Man;Woman': '',
}))

clean_2021['LastNewJob'] = missing_2021

clean_2021['LearnCodeFrom'] = df_2021['LearnCode'].apply(remove_spaces).apply(replace_multi({
    'School': 'School (i.e., University, College, etc)',
    'Other online resources (ex: videos, blogs, etc)': 'Other online resources (e.g., videos, blogs, forum)',
    'Online Forum': 'Other online resources (e.g., videos, blogs, forum)',
    'Other (please specify):': ''
}))

clean_2021['JobSeek'] = missing_2021

clean_2021['VersionControlSystem'] = missing_2021

clean_2021['LangPresent'] = df_2021['LanguageHaveWorkedWith'].apply(replace_multi({'Matlab': 'MATLAB'}))
clean_2021['LangFuture'] = df_2021['LanguageWantToWorkWith'].apply(replace_multi({'Matlab': 'MATLAB'}))

clean_2021['WebframePresent'] = df_2021['WebframeHaveWorkedWith'].apply(remove_spaces).apply(replace_multi({
    '': '',
}))
clean_2021['WebframeFuture'] = df_2021['WebframeWantToWorkWith'].apply(remove_spaces).apply(replace_multi({
    '': '',
}))

clean_2021['DatabasePresent'] = df_2021['DatabaseHaveWorkedWith'].apply(remove_spaces).apply(replace_multi({
    'IBM Db2': 'IBM DB2',
}))
clean_2021['DatabaseFuture'] = df_2021['DatabaseWantToWorkWith'].apply(remove_spaces).apply(replace_multi({
    'IBM Db2': 'IBM DB2',
}))

clean_2021['PlatformPresent'] = df_2021['PlatformHaveWorkedWith'].apply(remove_spaces)
clean_2021['PlatformFuture'] = df_2021['PlatformWantToWorkWith'].apply(remove_spaces)

clean_2021.head()

,Year,RespondentType,CodingActivities,Employment,Country,Student,Age,Education,OrgSize,DevType,...,JobSeek,VersionControlSystem,LangPresent,LangFuture,WebframePresent,WebframeFuture,DatabasePresent,DatabaseFuture,PlatformPresent,PlatformFuture
0,2021,dev,NaN,independent,Slovakia,no,25-34,secondary,20 to 99 employees,"Developer, mobile",...,NaN,NaN,C++;HTML/CSS;JavaScript;Objective-C;PHP;Swift,Swift,Laravel;Symfony,NaN,PostgreSQL;SQLite,SQLite,NaN,NaN
1,2021,stu,NaN,notwork,Netherlands,yes,18-24,bachelor,NaN,NaN,...,NaN,NaN,JavaScript;Python,NaN,Angular;Flask;Vue.js,NaN,PostgreSQL,NaN,NaN,NaN
2,2021,non-dev,NaN,notwork,Russian Federation,yes,18-24,bachelor,NaN,NaN,...,NaN,NaN,Assembly;C;Python;R;Rust,Julia;Python;Rust,Flask,Flask,SQLite,SQLite,Heroku,NaN
3,2021,dev,NaN,fulltime,Austria,no,35-44,master,100 to 499 employees,"Developer, front-end",...,NaN,NaN,JavaScript;TypeScript,JavaScript;TypeScript,Angular;jQuery,Angular;jQuery,NaN,NaN,NaN,NaN
4,2021,dev,NaN,independent,United Kingdom,no,25-34,master,"Just me - I am a freelancer, sole proprietor, ...","Developer, desktop or enterprise applications;...",...,NaN,NaN,Bash/Shell;HTML/CSS;Python;SQL,Bash/Shell;HTML/CSS;Python;SQL,Flask,Flask,Elasticsearch;PostgreSQL;Redis,Cassandra;Elasticsearch;PostgreSQL;Redis,NaN,NaN


In [182]:
clean_2022 = pd.DataFrame()
missing_2022 = pd.Series((np.nan for _ in range(len(df_2022))))

clean_2022['Year'] = pd.Series((2022 for _ in range(len(df_2022))))
clean_2022['RespondentType'] = df_2022['MainBranch'].replace(
    ['I am a developer by profession', 'I am learning to code',
     'I am not primarily a developer, but I write code sometimes as part of my work', 'I code primarily as a hobby',
     'I used to be a developer by profession, but no longer am', 'None of these'],
    ['dev', 'stu', 'non-dev', 'non-dev', 'ex-dev', 'non-dev']
)

clean_2022['CodingActivities'] = df_2022['CodingActivities'].apply(remove_spaces).apply(replace_multi({
    'Other (please specify):': ''
}))


def get_first(s: str | float) -> str | float:
    if not isinstance(s, str):
        return np.nan
    return s.split(';')[0]


clean_2022['Employment'] = df_2022['Employment'].apply(get_first).replace(
    ['Not employed, and not looking for work', 'Employed, full-time', 'Employed, part-time',
     'Independent contractor, freelancer, or self-employed', 'Not employed, but looking for work',
     'Retired', 'Student, full-time', 'Student, part-time', 'I prefer not to say'],
    ['notwork', 'fulltime', 'parttime', 'independent', 'searching', 'retired', 'notwork', 'parttime', np.nan],
)
clean_2022['Age'] = df_2022['Age'].replace(
    ['Prefer not to say', '25-34 years old', '18-24 years old', '35-44 years old', '45-54 years old',
     'Under 18 years old', '65 years or older', '55-64 years old'],
    [np.nan, '25-34', '18-24', '35-44', '45-54', '-17', '65-', '55-64']
)

clean_2022['Country'] = df_2022['Country'].replace(countries)

clean_2022['Student'] = df_2022['Employment'].apply(is_student_21)

clean_2022['Education'] = df_2022['EdLevel'].replace(
    ['Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
     "Some college/university study without earning a degree", "Bachelor’s degree (B.A., B.S., B.Eng., etc.)",
     'Other doctoral degree (Ph.D., Ed.D., etc.)', "Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",
     'Professional degree (JD, MD, etc.)', 'Primary/elementary school', 'I prefer not to answer',
     'I never completed any formal education', 'Associate degree (A.A., A.S., etc.)', 'Something else'],
    ['secondary', 'tertiary', 'bachelor', 'doctor', 'master', 'professional', 'primary', np.nan, 'none', 'assoc',
     np.nan],
)

clean_2022['OrgSize'] = df_2022['OrgSize'].replace(
    ['I don’t know'],
    [np.nan],
)

clean_2022['DevType'] = df_2022['DevType'].apply(replace_multi({
    'Other (please specify):': '',
    'Educator': 'Educator or academic',
    'Academic researcher': 'Educator or academic',
}))

clean_2022['YearsCode'] = df_2022['YearsCode'].apply(filter_year_19)
clean_2022['YearsCodePro'] = df_2022['YearsCodePro'].apply(filter_year_19)

clean_2022['JobSat'] = missing_2022

clean_2022['Currency'] = df_2022['Currency']
clean_2022['Salary'] = df_2022['ConvertedCompYearly'].apply(cap_salary)

clean_2022['CodeEditor'] = df_2022['NEWCollabToolsHaveWorkedWith']

clean_2022['Gender'] = df_2022['Gender'].apply(replace_multi({
    'Male': 'Man',
    'Female': 'Woman',
    'Non-binary, genderqueer, or gender non-conforming': '',
    'Or, in your own words:': '',
    'Prefer not to say': '',
    'Man;Woman': '',
    'Man;;Woman': '',
}))

clean_2022['LastNewJob'] = missing_2022

clean_2022['LearnCodeFrom'] = df_2022['LearnCode'].apply(replace_multi({
    'Other (please specify):': ''
}))

clean_2022['JobSeek'] = missing_2022

clean_2022['VersionControlSystem'] = missing_2022

clean_2022['LangPresent'] = df_2022['LanguageHaveWorkedWith'].apply(replace_multi({'Matlab': 'MATLAB'}))
clean_2022['LangFuture'] = df_2022['LanguageWantToWorkWith'].apply(replace_multi({'Matlab': 'MATLAB'}))

clean_2022['WebframePresent'] = df_2022['WebframeHaveWorkedWith'].apply(replace_multi({
    '': '',
}))
clean_2022['WebframeFuture'] = df_2022['WebframeWantToWorkWith'].apply(replace_multi({
    '': '',
}))

clean_2022['DatabasePresent'] = df_2022['DatabaseHaveWorkedWith'].apply(replace_multi({
    'IBM Db2': 'IBM DB2',
}))
clean_2022['DatabaseFuture'] = df_2022['DatabaseWantToWorkWith'].apply(replace_multi({
    'IBM Db2': 'IBM DB2',
}))

clean_2022['PlatformPresent'] = df_2022['PlatformHaveWorkedWith']
clean_2022['PlatformFuture'] = df_2022['PlatformWantToWorkWith']

clean_2022.head()

,Year,RespondentType,CodingActivities,Employment,Age,Country,Student,Education,OrgSize,DevType,...,JobSeek,VersionControlSystem,LangPresent,LangFuture,WebframePresent,WebframeFuture,DatabasePresent,DatabaseFuture,PlatformPresent,PlatformFuture
0,2022,non-dev,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022,dev,Hobby;Contribute to open-source projects,fulltime,NaN,Canada,no,NaN,NaN,NaN,...,NaN,NaN,JavaScript;TypeScript,Rust;TypeScript,NaN,NaN,NaN,NaN,NaN,NaN
2,2022,non-dev,Hobby,fulltime,25-34,United Kingdom,no,master,20 to 99 employees,Data scientist or machine learning specialist;...,...,NaN,NaN,C#;C++;HTML/CSS;JavaScript;Python,C#;C++;HTML/CSS;JavaScript;TypeScript,Angular.js,Angular;Angular.js,Microsoft SQL Server,Microsoft SQL Server,NaN,NaN
3,2022,dev,I don’t code outside of work,fulltime,35-44,Israel,no,bachelor,100 to 499 employees,"Developer, full-stack",...,NaN,NaN,C#;JavaScript;SQL;TypeScript,C#;SQL;TypeScript,ASP.NET;ASP.NET Core,ASP.NET;ASP.NET Core,Microsoft SQL Server,Microsoft SQL Server,NaN,NaN
4,2022,dev,Hobby,fulltime,25-34,United States,no,bachelor,20 to 99 employees,"Developer, front-end;Developer, full-stack;Dev...",...,NaN,NaN,C#;HTML/CSS;JavaScript;SQL;Swift;TypeScript,C#;Elixir;F#;Go;JavaScript;Rust;TypeScript,Angular;ASP.NET;ASP.NET Core ;jQuery;Node.js,Angular;ASP.NET Core ;Blazor;Node.js,Cloud Firestore;Elasticsearch;Microsoft SQL Se...,Cloud Firestore;Elasticsearch;Firebase Realtim...,Firebase;Microsoft Azure,Firebase;Microsoft Azure


In [261]:
joined = pd.concat([clean_2017, clean_2018, clean_2019, clean_2020, clean_2021, clean_2022])
final = pd.DataFrame()

final['Year'] = joined['Year'].astype('uint16')
final['Salary'] = joined['Salary'].astype('float32')
final['JobSat'] = joined['JobSat'].astype('Int8')
final['YearsCode'] = joined['YearsCode'].astype('Int8')
final['YearsCodePro'] = joined['YearsCodePro'].astype('Int8')
final['Age'] = joined['Age'].astype(
    pd.CategoricalDtype(['-17', '18-24', '25-34', '35-44', '45-54', '55-64', '65-'], True))
final['Education'] = joined['Education'].astype(pd.CategoricalDtype(
    ['none', 'primary', 'secondary', 'tertiary', 'assoc', 'bachelor', 'master', 'professional', 'doctor'], True))
final['OrgSize'] = joined['OrgSize'].astype(pd.CategoricalDtype(
    ['Just me - I am a freelancer, sole proprietor, etc.', '2 to 9 employees',
     '10 to 19 employees', '20 to 99 employees', '100 to 499 employees',
     '500 to 999 employees', '1,000 to 4,999 employees', '5,000 to 9,999 employees',
     '10,000 or more employees'], True))
final['LastNewJob'] = joined['LastNewJob'].astype(pd.CategoricalDtype(
    ['Less than a year ago', 'Between 1 and 2 years ago', 'Between 2 and 4 years ago', 'More than 4 years ago'], True))

for cat_col in ['Employment', 'RespondentType', 'JobSeek',
                'Gender', 'Student', 'Country']:
    final[cat_col] = joined[cat_col].replace([''], [np.nan]).astype('category')

for str_col in ['CodingActivities', 'DevType',
                'LearnCodeFrom', 'LangPresent', 'LangFuture']:
    final[str_col] = joined[str_col].replace([''], [np.nan]).astype('string')

# Dropped columns (irrelevant):

# 'Currency', 'WebframePresent', 'WebframeFuture', 'DatabasePresent', 'DatabaseFuture', 'PlatformPresent',
# 'PlatformFuture', 'CodeEditor', 'VersionControlSystem'

final.reset_index(drop=True, inplace=True)
final.head()

,Year,Salary,JobSat,YearsCode,YearsCodePro,Age,Education,OrgSize,LastNewJob,Employment,RespondentType,JobSeek,Gender,Student,Country,CodingActivities,DevType,LearnCodeFrom,LangPresent,LangFuture
0,2017,NaN,<NA>,2,<NA>,NaN,secondary,NaN,NaN,notwork,stu,maybe,Man,no,United States,Hobby;Contribute to open-source projects,<NA>,Online Courses or Certification;Other online r...,Swift,Swift
1,2017,NaN,<NA>,9,<NA>,NaN,tertiary,20 to 99 employees,NaN,parttime,stu,NaN,Man,yes,United Kingdom,Hobby;Contribute to open-source projects,<NA>,Online Courses or Certification;Hackathons (vi...,JavaScript;Python;Ruby;SQL,Java;Python;Ruby;SQL
2,2017,113750.0,5,20,20,NaN,bachelor,"10,000 or more employees",NaN,fulltime,dev,NaN,Man,no,United Kingdom,Hobby;Contribute to open-source projects,<NA>,"Other online resources (e.g., videos, blogs, f...",Java;PHP;Python,C;Python;Rust
3,2017,NaN,2,14,9,NaN,doctor,"10,000 or more employees",Between 2 and 4 years ago,fulltime,non-dev,yes,Man,no,United States,Hobby;Contribute to open-source projects,Data scientist or machine learning specialist,<NA>,MATLAB;Python;R;SQL,MATLAB;Python;R;SQL
4,2017,NaN,4,20,10,NaN,master,10 to 19 employees,NaN,fulltime,dev,NaN,NaN,no,Switzerland,Hobby,"Developer, mobile;Developer, game or graphics;...",<NA>,<NA>,<NA>


In [273]:
# Save to disk:

# final.to_parquet('SO_2_0.pq.gz', compression='gzip')